# analysis

## ToDo

- __Importing more than one data json__
- How to group by subjects models and movements
- Model specific confusion rate
- How to manage or represent all this data
- ...

## data structure

...

In [45]:
import numpy as np
import pandas as pd
import json, os

In [90]:
path_to_json = '/users/hieu/git/turing_test_walking/data'

json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
first_file = json_files.pop()
with open(first_file,'r') as f:
    file = json.loads(f.read())
data = pd.json_normalize(file)

In [92]:
first_file

'experiment_data_undefined_54q1vpsf6z80rza.json'

  ## parsing

In [ ]:
df = df[df['test_part'] == 'response'].reset_index()

df['button_pressed'] = df['button_pressed'].astype(int)
df['test_stimulus.position'] = df['test_stimulus.position'].astype(int)

df = df.rename(columns={"test_stimulus.left": "left",
                        "test_stimulus.right": "right",
                        "test_stimulus.att_check": "att_check",
                        "test_stimulus.position": "position",
                        "button_pressed":"pressed"})

In [77]:
df = df[['subject','rt','pressed','position',
    'left','right','att_check']]

In [78]:
df

,subject,rt,pressed,position,left,right,att_check
0,r6vg0soyf6xlcpt,852.0,1,1,bvh/vr_prediction_models/model(vcgpdm)-dataset...,bvh/return-bottle-hold-training2-lines.txt,False
1,r6vg0soyf6xlcpt,822.0,1,1,bvh/vr_prediction_models/model(dmp)-dataset(re...,bvh/return-bottle-training2-lines.txt,True
2,r6vg0soyf6xlcpt,1167.0,1,1,bvh/vr_prediction_models/model(dmp)-dataset(pa...,bvh/pass-bottle-training1-lines.txt,False
3,r6vg0soyf6xlcpt,855.0,1,1,bvh/vr_prediction_models/model(vcgpdm)-dataset...,bvh/return-bottle-hold-training3-lines.txt,False
4,r6vg0soyf6xlcpt,935.0,1,0,bvh/pass-bottle-hold-training0-lines.txt,bvh/vr_prediction_models/model(vcgpdm)-dataset...,False
...,...,...,...,...,...,...,...
105,r6vg0soyf6xlcpt,710.0,0,0,bvh/return-bottle-training2-lines.txt,bvh/vr_prediction_models/model(vcgpdm)-dataset...,False
106,r6vg0soyf6xlcpt,1247.0,1,1,bvh/vr_prediction_models/model(dmp)-dataset(pa...,bvh/pass-bottle-hold-training3-lines.txt,False
107,r6vg0soyf6xlcpt,1184.0,1,0,bvh/return-bottle-training3-lines.txt,bvh/vr_prediction_models/model(vcgpdm)-dataset...,False
108,r6vg0soyf6xlcpt,966.0,0,0,bvh/return-bottle-hold-training1-lines.txt,bvh/vr_prediction_models/model(dmp)-dataset(re...,False


In [49]:
# without copy() there is an error when replacing strings
d = df.copy()

In [50]:
# removing unnecessary strings
d['left'] = d['left'].str.replace('bvh/vr_prediction_models/','')
d['left'] = d['left'].str.replace('bvh/','')
d['left'] = d['left'].str.replace('/final-lines.txt','')
d['left'] = d['left'].str.replace('-lines.txt','')
d['right'] = d['right'].str.replace('bvh/vr_prediction_models/','')
d['right'] = d['right'].str.replace('bvh/','')
d['right'] = d['right'].str.replace('/final-lines.txt','')
d['right'] = d['right'].str.replace('-lines.txt','')

In [51]:
num = 0
model = []
for i in d['position']:
    if i == 1:
        model.append(d['left'][num])
    else:
        model.append(d['right'][num])
    num += 1
d['model'] = model

In [52]:
d

,subject,rt,pressed,position,left,right,att_check,model
0,r6vg0soyf6xlcpt,852.0,1,1,model(vcgpdm)-dataset(return-bottle-hold)-mode...,return-bottle-hold-training2,False,model(vcgpdm)-dataset(return-bottle-hold)-mode...
1,r6vg0soyf6xlcpt,822.0,1,1,model(dmp)-dataset(return-bottle)-npsi(90)-hol...,return-bottle-training2,True,model(dmp)-dataset(return-bottle)-npsi(90)-hol...
2,r6vg0soyf6xlcpt,1167.0,1,1,model(dmp)-dataset(pass-bottle)-npsi(70)-hold(1),pass-bottle-training1,False,model(dmp)-dataset(pass-bottle)-npsi(70)-hold(1)
3,r6vg0soyf6xlcpt,855.0,1,1,model(vcgpdm)-dataset(return-bottle-hold)-mode...,return-bottle-hold-training3,False,model(vcgpdm)-dataset(return-bottle-hold)-mode...
4,r6vg0soyf6xlcpt,935.0,1,0,pass-bottle-hold-training0,model(vcgpdm)-dataset(pass-bottle-hold)-mode(E...,False,model(vcgpdm)-dataset(pass-bottle-hold)-mode(E...
...,...,...,...,...,...,...,...,...
105,r6vg0soyf6xlcpt,710.0,0,0,return-bottle-training2,model(vcgpdm)-dataset(return-bottle)-mode(ELBO...,False,model(vcgpdm)-dataset(return-bottle)-mode(ELBO...
106,r6vg0soyf6xlcpt,1247.0,1,1,model(dmp)-dataset(pass-bottle-hold)-npsi(20)-...,pass-bottle-hold-training3,False,model(dmp)-dataset(pass-bottle-hold)-npsi(20)-...
107,r6vg0soyf6xlcpt,1184.0,1,0,return-bottle-training3,model(vcgpdm)-dataset(return-bottle)-mode(ELBO...,False,model(vcgpdm)-dataset(return-bottle)-mode(ELBO...
108,r6vg0soyf6xlcpt,966.0,0,0,return-bottle-hold-training1,model(dmp)-dataset(return-bottle-hold)-npsi(50...,False,model(dmp)-dataset(return-bottle-hold)-npsi(50...


## Analysis
### Individual confusion rate over all models
#### with all attention checks included

In [53]:
arr = list(zip(d['pressed'],d['position']))
right = 0
for i in arr:
    if i[0] == i[1]:
        right += 1
confusion_rate = (len(arr) - right)/len(arr)
print(confusion_rate)

0.33636363636363636


#### without attention checks

In [54]:
no_check = d[d['att_check']==False]

In [55]:
arr_no_check = list(zip(no_check['pressed'],no_check['position']))
right = 0
for i in arr_no_check:
    if i[0] == i[1]:
        right += 1
confusion_rate = (len(arr_no_check) - right)/len(arr_no_check)
print(confusion_rate)

0.37


### Checking attention
#### if confusion rate is zero -> good

In [56]:
check_att = d[d['att_check']==True]

In [57]:
arr_check = list(zip(check_att['pressed'],check_att['position']))
right = 0
for i in arr_check:
    if i[0] == i[1]:
        right += 1
confusion_rate = (len(arr_check) - right)/len(arr_check)
print(confusion_rate)

0.0


In [58]:
for i in d:
        print(d[i].shape)

(110,)
(110,)
(110,)
(110,)
(110,)
(110,)
(110,)
(110,)


In [85]:
df = []
for item in json_files: 
    with open(item,'r') as f: 
        load = json.loads(f.read())
        df.append(pd.json_normalize(load))

In [86]:
df

[         rt                                           stimulus button_pressed  \
 0    3025.0  <p>Welcome!<br> Please choose your language to...              1   
 1    2187.0                                                NaN            NaN   
 2     703.0                                     <p>Bereit?</p>              0   
 3       NaN               <div style="font-size:60px;">+</div>            NaN   
 4       NaN                                                NaN            NaN   
 ..      ...                                                ...            ...   
 442   966.0  <p>Welche Sequenz haben Sie als natürlicher em...              0   
 443   850.0                                     <p>Bereit?</p>              0   
 444     NaN               <div style="font-size:60px;">+</div>            NaN   
 445     NaN                                                NaN            NaN   
 446  2373.0  <p>Welche Sequenz haben Sie als natürlicher em...              1   
 
              